# 🚀 Retail Analytics Copilot - Complete Implementation
## Step-by-Step Code with Explanations

This notebook contains **all the code** from the project with detailed explanations for each step.

You can run each cell to see how everything works!

---
## 📦 Step 1: Install Dependencies

First, we need to install all required libraries.

In [ ]:
# Install all dependencies
# Run this cell once to set up your environment

!pip install dspy-ai>=2.4.0
!pip install langgraph>=0.1.0
!pip install langchain-core>=0.2.0
!pip install pydantic>=2.0.0
!pip install click>=8.1.7
!pip install rich>=13.7.0
!pip install numpy>=1.26.0
!pip install pandas>=2.2.0
!pip install scikit-learn>=1.3.0
!pip install rank-bm25>=0.2.2

print("✅ All dependencies installed!")

---
## 🗄️ Step 2: Database Tool (SQLite Access)

This class handles all database operations:
- Getting the schema (table structure)
- Running SQL queries
- Error handling

In [1]:
import sqlite3
from typing import List, Dict, Any, Optional

class SQLiteTool:
    """Tool for accessing SQLite database"""
    
    def __init__(self, db_path: str):
        """
        Initialize the database tool.
        
        Args:
            db_path: Path to the SQLite database file
        """
        self.db_path = db_path

    def get_schema(self) -> str:
        """
        Get the database schema (table names and columns).
        This is used by the AI to understand what's in the database.
        
        Returns:
            String describing all tables and their columns
        """
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        # Get list of all tables
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()
        
        schema_str = ""
        for table in tables:
            table_name = table[0]
            # Get column information for each table
            cursor.execute(f"PRAGMA table_info('{table_name}');")
            columns = cursor.fetchall()
            
            schema_str += f"Table: {table_name}\n"
            for col in columns:
                # col[1] is column name, col[2] is data type
                schema_str += f"  - {col[1]} ({col[2]})\n"
            schema_str += "\n"
            
        conn.close()
        return schema_str

    def execute_query(self, query: str) -> Dict[str, Any]:
        """
        Execute a SQL query and return results.
        
        Args:
            query: SQL query string
            
        Returns:
            Dictionary with columns, rows, and any error
        """
        try:
            conn = sqlite3.connect(self.db_path)
            cursor = conn.cursor()
            cursor.execute(query)
            
            # Get column names from the result
            columns = [description[0] for description in cursor.description]
            rows = cursor.fetchall()
            
            conn.close()
            return {
                "columns": columns,
                "rows": rows,
                "error": None
            }
        except Exception as e:
            # If there's an error, return it
            return {
                "columns": [],
                "rows": [],
                "error": str(e)
            }

# Test the tool
print("✅ SQLiteTool class created!")
print("\nExample usage:")
print("  db_tool = SQLiteTool('data/northwind.sqlite')")
print("  schema = db_tool.get_schema()")
print("  result = db_tool.execute_query('SELECT * FROM Products LIMIT 5')")

✅ SQLiteTool class created!

Example usage:
  db_tool = SQLiteTool('data/northwind.sqlite')
  schema = db_tool.get_schema()
  result = db_tool.execute_query('SELECT * FROM Products LIMIT 5')


### Test the Database Tool

In [2]:
# Create the tool instance
db_tool = SQLiteTool('data/northwind.sqlite')

# Get and display schema
schema = db_tool.get_schema()
print("Database Schema (first 800 characters):")
print(schema[:800])

# Run a test query
result = db_tool.execute_query("SELECT COUNT(*) as product_count FROM Products")
print("\n\nTest Query: How many products?")
print(f"Columns: {result['columns']}")
print(f"Result: {result['rows']}")
print(f"Error: {result['error']}")

Database Schema (first 800 characters):
Table: Categories
  - CategoryID (INTEGER)
  - CategoryName (TEXT)
  - Description (TEXT)
  - Picture (BLOB)

Table: sqlite_sequence
  - name ()
  - seq ()

Table: CustomerCustomerDemo
  - CustomerID (TEXT)
  - CustomerTypeID (TEXT)

Table: CustomerDemographics
  - CustomerTypeID (TEXT)
  - CustomerDesc (TEXT)

Table: Customers
  - CustomerID (TEXT)
  - CompanyName (TEXT)
  - ContactName (TEXT)
  - ContactTitle (TEXT)
  - Address (TEXT)
  - City (TEXT)
  - Region (TEXT)
  - PostalCode (TEXT)
  - Country (TEXT)
  - Phone (TEXT)
  - Fax (TEXT)

Table: Employees
  - EmployeeID (INTEGER)
  - LastName (TEXT)
  - FirstName (TEXT)
  - Title (TEXT)
  - TitleOfCourtesy (TEXT)
  - BirthDate (DATE)
  - HireDate (DATE)
  - Address (TEXT)
  - City (TEXT)
  - Region (TEXT)
  - PostalCode (TEXT)
  - Coun


Test Query: How many products?
Columns: ['product_count']
Result: [(77,)]
Error: None


---
## 📚 Step 3: Document Retriever (BM25 Search)

This class searches through our documentation:
- Loads markdown files
- Splits them into chunks
- Uses BM25 algorithm to find relevant chunks

In [ ]:
import os
from typing import List, Dict, Any
from rank_bm25 import BM25Okapi
import glob

class Retriever:
    """Document retriever using BM25 search algorithm"""
    
    def __init__(self, docs_dir: str):
        """
        Initialize the retriever by loading all documents.
        
        Args:
            docs_dir: Directory containing markdown files
        """
        self.docs_dir = docs_dir
        self.chunks: List[Dict[str, Any]] = []
        self.bm25 = None
        self._load_documents()

    def _load_documents(self):
        """
        Load all markdown files and split into chunks.
        Each chunk gets a unique ID for citation.
        """
        # Find all .md files
        file_paths = glob.glob(os.path.join(self.docs_dir, "*.md"))
        
        for file_path in file_paths:
            filename = os.path.basename(file_path)
            
            # Read file content
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read()
                
            # Split by paragraphs (double newlines)
            paragraphs = [p.strip() for p in content.split('\n\n') if p.strip()]
            
            # Create chunks with IDs
            for j, para in enumerate(paragraphs):
                chunk_id = f"{filename.replace('.md', '')}::chunk{j}"
                self.chunks.append({
                    "id": chunk_id,
                    "content": para,
                    "source": filename
                })

        # Initialize BM25 with tokenized corpus
        # BM25 works by comparing word frequencies
        tokenized_corpus = [chunk["content"].lower().split() for chunk in self.chunks]
        self.bm25 = BM25Okapi(tokenized_corpus)

    def retrieve(self, query: str, k: int = 3) -> List[Dict[str, Any]]:
        """
        Find the top-k most relevant chunks for a query.
        
        Args:
            query: Search query
            k: Number of results to return
            
        Returns:
            List of chunks with scores
        """
        if not self.chunks:
            return []
            
        # Tokenize query
        tokenized_query = query.lower().split()
        
        # Get BM25 scores for all chunks
        scores = self.bm25.get_scores(tokenized_query)
        
        # Get indices of top-k chunks
        top_n = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:k]
        
        # Return top chunks with scores
        results = []
        for i in top_n:
            results.append({
                **self.chunks[i],
                "score": float(scores[i])
            })
            
        return results

print("✅ Retriever class created!")

### Test the Retriever

In [ ]:
# Create retriever
retriever = Retriever('docs')

print(f"Loaded {len(retriever.chunks)} document chunks")
print("\nChunk IDs:")
for chunk in retriever.chunks[:5]:
    print(f"  - {chunk['id']}")

# Test search
query = "return policy beverages"
results = retriever.retrieve(query, k=2)

print(f"\n\nSearch Results for: '{query}'\n")
for i, result in enumerate(results, 1):
    print(f"{i}. Chunk ID: {result['id']}")
    print(f"   Score: {result['score']:.3f}")
    print(f"   Content: {result['content'][:100]}...\n")

---
## 🧠 Step 4: DSPy Signatures (AI Module Definitions)

DSPy Signatures define what each AI module should do:
- **Router**: Classify questions (rag/sql/hybrid)
- **Planner**: Extract constraints from docs
- **SQL Generator**: Write SQL queries
- **Synthesizer**: Create final answers

In [3]:
import dspy
from typing import List, Optional

# Signature 1: Router - Classifies the question type
class Router(dspy.Signature):
    """Classify the user question into one of: 'rag', 'sql', 'hybrid'."""
    
    question = dspy.InputField(desc="The user's question about retail analytics.")
    classification = dspy.OutputField(desc="One of: 'rag', 'sql', 'hybrid'.")

# Signature 2: Planner - Extracts information from documents
class Planner(dspy.Signature):
    """Extract constraints, date ranges, and entities from the question and docs."""
    
    question = dspy.InputField()
    retrieved_docs = dspy.InputField(desc="Relevant document chunks.")
    plan = dspy.OutputField(desc="Step-by-step plan including date ranges, KPIs, and entities.")

# Signature 3: SQL Generator - Writes SQL queries
class GenerateSQL(dspy.Signature):
    """Generate a SQLite query based on the question, schema, and plan."""
    
    question = dspy.InputField()
    plan = dspy.InputField(desc="Execution plan with constraints and entities.")
    db_schema = dspy.InputField(desc="Schema of the available tables.")
    sql_query = dspy.OutputField(desc="The SQLite query to answer the question.")

# Signature 4: Synthesizer - Creates final answer
class SynthesizeAnswer(dspy.Signature):
    """Synthesize a final answer based on the question, SQL results, and retrieved documents."""
    
    question = dspy.InputField()
    sql_query = dspy.InputField(desc="The SQL query executed, if any.")
    sql_result = dspy.InputField(desc="The result of the SQL query, if any.")
    retrieved_docs = dspy.InputField(desc="Relevant document chunks from the knowledge base.")
    format_hint = dspy.InputField(desc="The expected format of the answer (e.g., int, float, list[dict]).")
    
    final_answer = dspy.OutputField(desc="The answer matching the format hint.")
    explanation = dspy.OutputField(desc="A brief explanation (<= 2 sentences).")
    citations = dspy.OutputField(desc="List of DB tables and doc chunk IDs used.")

print("✅ All DSPy Signatures defined!")

✅ All DSPy Signatures defined!


### DSPy Modules (Wrap Signatures with ChainOfThought)

In [ ]:
# Modules wrap signatures with Chain of Thought reasoning
# This makes the AI "think" before answering

class CoT_Router(dspy.Module):
    """Router module with Chain of Thought"""
    def __init__(self):
        super().__init__()
        self.prog = dspy.ChainOfThought(Router)
    
    def forward(self, question):
        return self.prog(question=question)

class CoT_Planner(dspy.Module):
    """Planner module with Chain of Thought"""
    def __init__(self):
        super().__init__()
        self.prog = dspy.ChainOfThought(Planner)
    
    def forward(self, question, retrieved_docs):
        return self.prog(question=question, retrieved_docs=retrieved_docs)

class CoT_SQL(dspy.Module):
    """SQL Generator module with Chain of Thought"""
    def __init__(self):
        super().__init__()
        self.prog = dspy.ChainOfThought(GenerateSQL)
    
    def forward(self, question, db_schema, plan=""):
        return self.prog(question=question, db_schema=db_schema, plan=plan)

class CoT_Synthesizer(dspy.Module):
    """Synthesizer module with Chain of Thought"""
    def __init__(self):
        super().__init__()
        self.prog = dspy.ChainOfThought(SynthesizeAnswer)
    
    def forward(self, question, sql_query, sql_result, retrieved_docs, format_hint):
        return self.prog(
            question=question,
            sql_query=sql_query,
            sql_result=sql_result,
            retrieved_docs=retrieved_docs,
            format_hint=format_hint
        )

print("✅ All DSPy Modules created!")

---
## 🕸️ Step 5: LangGraph Workflow (The Agent)

This is the main agent that connects everything:
- 7 nodes (Router, Retriever, Planner, SQL Gen, Executor, Synthesizer, Repair)
- Conditional edges based on classification and errors
- State management

In [ ]:
from typing import TypedDict, Any, Union
from langgraph.graph import StateGraph, END

# Define the state that flows through the graph
class AgentState(TypedDict):
    question: str
    format_hint: str
    classification: str
    plan: str
    sql_query: Optional[str]
    sql_result: Optional[Dict[str, Any]]
    retrieved_docs: List[Dict[str, Any]]
    final_answer: Any
    explanation: str
    citations: List[str]
    confidence: float
    error: Optional[str]
    repair_count: int

print("✅ AgentState defined!")

In [ ]:
class HybridAgent:
    """Main agent that combines RAG and SQL"""
    
    def __init__(self, db_path: str, docs_dir: str):
        """
        Initialize the agent with database and documents.
        
        Args:
            db_path: Path to SQLite database
            docs_dir: Directory with markdown documents
        """
        # Initialize tools
        self.db_tool = SQLiteTool(db_path)
        self.retriever = Retriever(docs_dir)
        self.schema = self.db_tool.get_schema()
        
        # Initialize DSPy modules
        self.router = CoT_Router()
        self.planner = CoT_Planner()
        self.sql_gen = CoT_SQL()
        
        # Load optimized SQL generator if available
        if os.path.exists("agent/optimized_sql_gen.json"):
            print("Loading optimized SQL generator...")
            self.sql_gen.load("agent/optimized_sql_gen.json")
            
        self.synthesizer = CoT_Synthesizer()
        
        # Build the graph
        self.app = self.build_graph()
    
    def build_graph(self):
        """Build the LangGraph workflow with 7 nodes"""
        workflow = StateGraph(AgentState)
        
        # Add all 7 nodes
        workflow.add_node("router", self.route_question)
        workflow.add_node("retriever", self.retrieve_docs)
        workflow.add_node("planner", self.plan_execution)
        workflow.add_node("sql_generator", self.generate_sql)
        workflow.add_node("executor", self.execute_sql)
        workflow.add_node("synthesizer", self.synthesize_answer)
        workflow.add_node("repair", self.repair_action)
        
        # Set entry point
        workflow.set_entry_point("router")
        
        # Add conditional edges based on classification
        workflow.add_conditional_edges(
            "router",
            self.decide_route,
            {
                "rag": "retriever",
                "sql": "sql_generator",
                "hybrid": "retriever"
            }
        )
        
        # From retriever, always go to planner
        workflow.add_conditional_edges(
            "retriever",
            lambda state: "planner",
            {"planner": "planner"}
        )
        
        # From planner, decide based on classification
        workflow.add_conditional_edges(
            "planner",
            lambda state: "sql_generator" if state["classification"] in ["sql", "hybrid"] else "synthesizer",
            {
                "sql_generator": "sql_generator",
                "synthesizer": "synthesizer"
            }
        )
        
        # From SQL generator to executor
        workflow.add_edge("sql_generator", "executor")
        
        # From executor, check for errors
        workflow.add_conditional_edges(
            "executor",
            self.check_execution,
            {
                "synthesizer": "synthesizer",
                "repair": "repair"
            }
        )
        
        # Repair loop (max 2 retries)
        workflow.add_conditional_edges(
            "repair",
            lambda state: "sql_generator" if state["repair_count"] <= 2 else "synthesizer",
            {
                "sql_generator": "sql_generator",
                "synthesizer": "synthesizer"
            }
        )
        
        # Final node
        workflow.add_edge("synthesizer", END)
        
        return workflow.compile()

    # Node functions below...
    def route_question(self, state: AgentState):
        """Node 1: Classify the question"""
        print(f"Routing question: {state['question']}")
        try:
            pred = self.router(question=state['question'])
            classification = pred.classification.lower() if hasattr(pred, 'classification') else "hybrid"
            if classification not in ["rag", "sql", "hybrid"]:
                classification = "hybrid"
            return {"classification": classification}
        except Exception as e:
            print(f"Router error: {e}, defaulting to hybrid")
            return {"classification": "hybrid"}

    def decide_route(self, state: AgentState):
        """Decision function for routing"""
        return state["classification"]

    def retrieve_docs(self, state: AgentState):
        """Node 2: Search documents"""
        print("Retrieving docs...")
        docs = self.retriever.retrieve(state["question"])
        return {"retrieved_docs": docs}

    def plan_execution(self, state: AgentState):
        """Node 3: Extract constraints from docs"""
        print("Planning execution...")
        try:
            retrieved_docs = str(state.get("retrieved_docs", []))
            pred = self.planner(question=state["question"], retrieved_docs=retrieved_docs)
            plan = pred.plan if hasattr(pred, 'plan') else str(pred)
            return {"plan": plan}
        except Exception as e:
            print(f"Planner error: {e}, using basic plan")
            return {"plan": f"Answer the question: {state['question']}"}

    def generate_sql(self, state: AgentState):
        """Node 4: Generate SQL query"""
        print("Generating SQL...")
        try:
            pred = self.sql_gen(
                question=state["question"], 
                db_schema=self.schema, 
                plan=state.get("plan", "")
            )
            sql_query = pred.sql_query if hasattr(pred, 'sql_query') else str(pred)
            return {"sql_query": sql_query}
        except Exception as e:
            print(f"SQL generation error: {e}")
            return {"sql_query": None, "error": f"SQL generation failed: {str(e)}"}

    def execute_sql(self, state: AgentState):
        """Node 5: Execute SQL query"""
        print(f"Executing SQL: {state['sql_query']}")
        result = self.db_tool.execute_query(state["sql_query"])
        error = result.get("error")
        return {"sql_result": result, "error": error}

    def check_execution(self, state: AgentState):
        """Check if SQL execution succeeded"""
        if state.get("error"):
            print(f"Execution Error: {state['error']}")
            return "repair"
        return "synthesizer"

    def repair_action(self, state: AgentState):
        """Node 6: Repair failed SQL"""
        print(f"Repairing... Attempt {state['repair_count'] + 1}")
        new_plan = state.get("plan", "") + f"\nPrevious SQL failed with error: {state['error']}. Fix the SQL."
        return {
            "repair_count": state["repair_count"] + 1,
            "plan": new_plan,
            "error": None
        }

    def synthesize_answer(self, state: AgentState):
        """Node 7: Create final answer"""
        print("Synthesizing answer...")
        
        sql_query = state.get("sql_query", "")
        sql_result = str(state.get("sql_result", ""))
        retrieved_docs = str(state.get("retrieved_docs", []))
        
        try:
            pred = self.synthesizer(
                question=state["question"],
                sql_query=sql_query,
                sql_result=sql_result,
                retrieved_docs=retrieved_docs,
                format_hint=state["format_hint"]
            )
            
            final_answer = pred.final_answer if hasattr(pred, 'final_answer') else None
            explanation = pred.explanation if hasattr(pred, 'explanation') else "Generated answer"
            citations = pred.citations if hasattr(pred, 'citations') else []
        except Exception as e:
            print(f"Synthesizer error: {e}, using fallback")
            final_answer = "Error generating answer"
            explanation = f"Synthesis failed: {str(e)}"
            citations = []
        
        # Type conversion
        try:
            if state["format_hint"] == "int":
                final_answer = int(float(str(final_answer).strip()))
            elif state["format_hint"] == "float":
                final_answer = float(str(final_answer).strip())
        except:
            pass 
            
        if isinstance(citations, str):
            citations = [c.strip() for c in citations.split(',')]
            
        # Calculate confidence
        confidence = 1.0
        if state.get("repair_count", 0) > 0:
            confidence -= (0.2 * state["repair_count"])
        if not citations:
            confidence -= 0.2
        if confidence < 0:
            confidence = 0.0
            
        return {
            "final_answer": final_answer,
            "explanation": explanation,
            "citations": citations,
            "confidence": round(confidence, 2)
        }
    
    def invoke(self, state):
        """Run the agent"""
        return self.app.invoke(state)

print("✅ HybridAgent class created!")

---
## 🎓 Step 6: DSPy Optimization (Training)

This code optimizes the SQL generator using examples.

In [ ]:
from dspy.teleprompt import BootstrapFewShot
import json

def optimize_sql_generator():
    """
    Optimize the SQL generator using training examples.
    This creates a better version with few-shot examples.
    """
    # Define validation metric
    def validate_sql(example, pred, trace=None):
        """Check if SQL is valid"""
        sql = pred.sql_query
        return sql is not None and len(sql) > 10 and "SELECT" in sql.upper()

    # Setup DSPy with local model
    lm = dspy.LM(model='ollama/phi3.5:3.8b-mini-instruct-q4_K_M', 
                 api_base='http://localhost:11434', 
                 max_tokens=1000)
    dspy.settings.configure(lm=lm)
    
    # Load training examples
    with open('train_examples.json', 'r') as f:
        raw_data = json.load(f)
    
    # Get database schema
    db_tool = SQLiteTool("data/northwind.sqlite")
    schema = db_tool.get_schema()
    
    # Create training set
    trainset = []
    for item in raw_data:
        trainset.append(dspy.Example(
            question=item['question'],
            db_schema=schema,
            sql_query=item['sql_query']
        ).with_inputs('question', 'db_schema'))
        
    # Create optimizer
    teleprompter = BootstrapFewShot(
        metric=validate_sql, 
        max_bootstrapped_demos=4, 
        max_labeled_demos=4
    )
    
    print("Compiling (optimizing) CoT_SQL...")
    optimized_sql_gen = teleprompter.compile(CoT_SQL(), trainset=trainset)
    
    # Save
    optimized_sql_gen.save("agent/optimized_sql_gen.json")
    print("Optimization complete. Saved to agent/optimized_sql_gen.json")
    
    return optimized_sql_gen

print("✅ Optimization function defined!")
print("\nTo run optimization:")
print("  optimized_sql_gen = optimize_sql_generator()")

---
## 🚀 Step 7: Running the Complete Agent

Now we can run the full agent on test questions!

In [ ]:
def run_agent_on_questions(batch_file: str, output_file: str):
    """
    Run the agent on a batch of questions.
    
    Args:
        batch_file: Path to JSONL file with questions
        output_file: Path to save results
    """
    import json
    import logging
    
    # Setup logging
    logging.basicConfig(
        filename='agent_trace.log',
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s'
    )
    
    # Setup DSPy LM
    try:
        lm = dspy.LM(model='ollama/phi3.5:3.8b-mini-instruct-q4_K_M', 
                     api_base='http://localhost:11434', 
                     max_tokens=1000)
    except Exception as e:
        print(f"Error: {e}")
        print("Make sure Ollama is running: ollama serve")
        print("And model is pulled: ollama pull phi3.5:3.8b-mini-instruct-q4_K_M")
        return
        
    dspy.settings.configure(lm=lm)
    
    # Initialize agent
    agent = HybridAgent(
        db_path="data/northwind.sqlite",
        docs_dir="docs"
    )
    
    results = []
    
    # Process each question
    with open(batch_file, 'r') as f:
        for line in f:
            item = json.loads(line)
            print(f"\nProcessing: {item['id']}")
            
            # Create initial state
            initial_state = {
                "question": item["question"],
                "format_hint": item["format_hint"],
                "classification": "",
                "plan": "",
                "sql_query": None,
                "sql_result": None,
                "retrieved_docs": [],
                "final_answer": None,
                "explanation": "",
                "citations": [],
                "confidence": 0.0,
                "error": None,
                "repair_count": 0
            }
            
            # Run agent
            final_state = agent.invoke(initial_state)
            
            logging.info(f"Processed {item['id']}. Final Answer: {final_state['final_answer']}")
            
            # Create output
            output = {
                "id": item["id"],
                "final_answer": final_state["final_answer"],
                "sql": final_state["sql_query"] if final_state["sql_query"] else "",
                "confidence": final_state.get("confidence", 0.0),
                "explanation": final_state["explanation"],
                "citations": final_state["citations"]
            }
            results.append(output)
            
    # Write results
    with open(output_file, 'w') as f:
        for res in results:
            f.write(json.dumps(res) + "\n")
            
    print(f"\n✅ Done! Results written to {output_file}")
    return results

print("✅ Run function defined!")
print("\nTo run the agent:")
print("  results = run_agent_on_questions('sample_questions_hybrid_eval.jsonl', 'outputs_hybrid.jsonl')")

---
## 📊 Step 8: Example Usage

Here's how to use everything together:

In [ ]:
# Example: Process a single question
print("""To process a single question:

1. Setup DSPy with Ollama:
   lm = dspy.LM(model='ollama/phi3.5:3.8b-mini-instruct-q4_K_M', 
                api_base='http://localhost:11434', max_tokens=1000)
   dspy.settings.configure(lm=lm)

2. Create agent:
   agent = HybridAgent('data/northwind.sqlite', 'docs')

3. Run on a question:
   state = {
       "question": "What was the revenue in June 1997?",
       "format_hint": "float",
       "classification": "",
       "plan": "",
       "sql_query": None,
       "sql_result": None,
       "retrieved_docs": [],
       "final_answer": None,
       "explanation": "",
       "citations": [],
       "confidence": 0.0,
       "error": None,
       "repair_count": 0
   }
   result = agent.invoke(state)
   print(result['final_answer'])
""")

---
## 🎯 Summary

### What We Built:

1. **SQLiteTool** - Database access
2. **Retriever** - Document search with BM25
3. **DSPy Signatures** - AI module definitions
4. **DSPy Modules** - Chain of Thought wrappers
5. **HybridAgent** - 7-node LangGraph workflow
6. **Optimization** - BootstrapFewShot training
7. **Runner** - Batch processing

### How It Works:

```
Question → Router → Retriever → Planner → SQL Gen → Executor → Synthesizer → Answer
                                              ↓ error ↓
                                             Repair (x2)
```

### To Run:

```bash
# 1. Make sure Ollama is running
ollama serve

# 2. Pull the model
ollama pull phi3.5:3.8b-mini-instruct-q4_K_M

# 3. Run the notebook cells in order!
```

**You now have the complete implementation with explanations!** 🎉